In [ ]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

import networkx as nx
from google.colab import files

In [ ]:
! git clone https://github.com/ContentSide/RezoJDM-SDS.git

fatal: destination path 'RezoJDM-SDS' already exists and is not an empty directory.


In [ ]:
nodes = pd.read_csv("/content/RezoJDM-SDS/datasets/Filtered_Nodes_Edges/nodes.csv", index_col=False)
relations = pd.read_csv("/content/RezoJDM-SDS/datasets/Filtered_Nodes_Edges/relations.csv", index_col=False)

In [ ]:
nodes_out_degree = {}
nodes_in_degree = {}
nodes_degree = {}

for _, row in relations.iterrows():
    
    if row.n1 in nodes_out_degree:
        nodes_out_degree[row.n1]+=1
    else:
        nodes_out_degree[row.n1]=1


    if row.n2 in nodes_in_degree:
        nodes_in_degree[row.n2]+=1
    else:
        nodes_in_degree[row.n2]=1


    if row.n1 in nodes_degree:
        nodes_degree[row.n1]+=1
    else:
        nodes_degree[row.n1]=1


    if row.n2 in nodes_degree:
        nodes_degree[row.n2]+=1
    else:
        nodes_degree[row.n2]=1


In [ ]:
values=[]
keys=[]

for key, value in nodes_degree.items():
    values.append(value)
    keys.append(key)
    if value == 15818:
        print(key)

116477


In [ ]:
len(relations["t"].unique())

113

In [ ]:
min(values), max(values), np.mean(values)

(1, 15818, 42.73066440725835)

In [ ]:
nodes[nodes["eid"]==116477]

,eid,n,t,w
35977,116477,personne,1,9834


In [ ]:
#Entities connected to less than 15 other nodes (degree(node)<15) were removed.

nodes_to_keep=[]
for key, value in nodes_degree.items():
    if value >= 45:
        nodes_to_keep.append(key)

relations_filter1 = relations[relations["n1"].isin(nodes_to_keep) & relations["n2"].isin(nodes_to_keep)].reset_index()

In [ ]:
len(nodes_to_keep)

15746

In [ ]:
relations_filter1.shape

(887654, 6)

In [ ]:
len(relations_filter1["t"].unique())

106

In [ ]:
#Relationships that appeared less than 100 times in the graph were also deleted.

relation_freq={}

for _, row in relations_filter1.iterrows():
    
    if row.t in relation_freq:
        relation_freq[row.t]+=1
    else:
        relation_freq[row.t]=1

In [ ]:
relations_to_keep=[]
for key, value in relation_freq.items():
    if value >= 100:
        relations_to_keep.append(key)

relations_filter2 = relations_filter1[relations_filter1["t"].isin(relations_to_keep)].reset_index()

In [ ]:
relations_filter1.shape, relations_filter2.shape, len(relations_filter2["t"].unique())

((887654, 6), (886393, 7), 54)

In [ ]:
# pairs of symmetric relationship types
symmetic_reltypes = [
  (21,20),
  (53,54),
  (50,51),
  (41,42),
  (31,30),
  (32,115),
  (39,40),
  (14,26),
  (16,25),
  (3,27),
  (73,74),
  (55,56),
  (13,24),
  (6,8),
  (17,23),
  (15,28),
  (1,2000),
  (9,10),
]

_relations_to_delete = set()

error = False

for n, reltype_pair in enumerate(symmetic_reltypes):
  print("Handling reltype pair: {} ({}/{})\n\n".format(reltype_pair, n+1, len(symmetic_reltypes)))
  _edges = relations_filter2[relations_filter2["t"].isin(reltype_pair)]
  for _, _edge in _edges.iterrows():
    if _edge["t"] == reltype_pair[0]:
      other_type = reltype_pair[1]
    elif _edge["t"] == reltype_pair[1]:
      other_type = reltype_pair[0]
    symmetric_rel = _edges[(_edges["t"] == other_type) & (_edges["n1"] == _edge["n2"]) & (_edges["n2"] == _edge["n1"])]
    if symmetric_rel.shape[0] == 0:
      continue
    elif symmetric_rel.shape[0] > 1:
    test = (_edge["w"] > symmetric_rel["w"]).values[0]
    _relations_to_delete.add(_edge["rid"] if test else symmetric_rel["rid"].values[0])  

In [ ]:
print("number of relations to delete:\n", len(_relations_to_delete))

In [ ]:
relations_filter3 = relations_filter2[~relations_filter2["rid"].isin(_relations_to_delete)]
relations_filter2.shape, relations_filter3.shape

In [ ]:
relations_filter2.shape[0] - relations_filter3.shape[0]

In [ ]:
# Let's make sure every relationship type still has >=100 examples. If not, remove it.

_relation_freq={}

for _, _row in relations_filter3.iterrows():
    
    if _row.t in _relation_freq:
        _relation_freq[_row.t]+=1
    else:
        _relation_freq[_row.t]=1

for _reltype, _relfreq in _relation_freq.items():
  if _relfreq < 100:
    print(f"WARNING! Relationship type {_reltype} has {_relfreq}<100 examples")

In [ ]:
relations_to_keep=[]
for key, value in _relation_freq.items():
    if value >= 100:
        relations_to_keep.append(key)

relations_filter4 = relations_filter3[relations_filter3["t"].isin(relations_to_keep)]
relations_filter3.shape, relations_filter4.shape

In [ ]:
relations_filter3.shape[0] - relations_filter4.shape[0]

In [ ]:
relations_filter2.shape, relations_filter4.shape, len(relations_filter2["t"].unique()), len(relations_filter4["t"].unique())

In [ ]:
#code to find components of the graph. The largest subgraph will be selected.

def get_sub_disconnected_graphs(relations):
    G = nx.from_pandas_edgelist(relations, source='n1', target='n2', edge_attr='t')
    largest_component = max(nx.connected_components(G), key=len)
    G2 = G.subgraph(largest_component)
    return len(list(nx.connected_components(G)))

In [ ]:
get_sub_disconnected_graphs(relations), get_sub_disconnected_graphs(relations_filter2), get_sub_disconnected_graphs(relations_filter4)

(1, 1, 1)

In [ ]:
relations = relations_filter4[["n1","n2","t","w"]]
relations.shape , len(relations["t"].unique())

((886393, 4), 54)

In [ ]:
# relations_info = {}

# for _, row in relations.iterrows():
#     start = min(row.n1,row.n2)
#     end = max(row.n1,row.n2)

#     link = str(start) + "-" + str(end)

#     if link in relations_info:
#         relations_info[link].append(row)
#     else:
#         relations_info[link] = [row]

In [ ]:
def get_subset_triplet(relations, N):

    relations_ = relations.copy()
    count = 0


    while count < N:

        rand_index = np.random.randint(relations_.shape[0], size=1)[0]
        relations_dropped = relations_.drop(rand_index)

        condition1 = relations_.loc[rand_index]["n1"] in relations_dropped["n1"]
        condition2 = relations_.loc[rand_index]["n2"] in relations_dropped["n2"]    

        if condition1 & condition2:

            if count == 0:
                relations_new = relations_.loc[rand_index].to_frame().T
            else:
                relations_new = relations_new.append(relations_.loc[rand_index], ignore_index=True)

            relations_ = relations_dropped.reset_index()[["n1","n2","t","w"]]
            count+=1

            if count % 1000 == 0:
                print(count)

    return relations_new, relations_

In [ ]:
rel_new, rel_old = get_subset_triplet(relations, 10)
rel_new.shape, rel_old.shape, relations.shape

((10, 4), (886383, 4), (886393, 4))

In [ ]:
total = 272 + 17 + 20
272/total, 17/total, 20/total

(0.8802588996763754, 0.05501618122977346, 0.06472491909385113)

In [ ]:
N = 832083
N*0.1 , N*0.15 , N*0.75

(88639.3, 132958.94999999998, 664794.75)

In [ ]:
triplets_dev, triplets_rest = get_subset_triplet(relations, 83209)
triplets_dev.to_csv("triplets_dev.csv", index=False)
triplets_rest.to_csv("triplets_rest.csv", index=False)
files.download("triplets_dev.csv")
files.download("triplets_rest.csv")

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
triplets_test, triplets_train = get_subset_triplet(triplets_rest, 83209)
triplets_train.to_csv("triplets_train.csv", index=False)
triplets_test.to_csv("triplets_test.csv", index=False)
files.download("triplets_train.csv")
files.download("triplets_test.csv")

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>